In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from starfish.core.structured_llm import StructuredLLM
from starfish.common.env_loader import load_env_file

load_env_file()

2025-04-04 00:10:15 | INFO     | starfish.common.env_loader | env_loader.py:54 | Loaded 9 environment variables from /Users/zhengisamazing/1.python_dir/starfish/.env


True

In [3]:
first_llm = StructuredLLM(
    model_name="openai/gpt-4o-mini",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
    model_kwargs={"temperature": 0.7},
)

first_response = await first_llm.run(city_name="New York")
first_response.data

[{'question': 'What is the population of New York City?',
  'answer': 'As of 2023, New York City has an estimated population of over 8.6 million people.'},
 {'question': 'What is New York City commonly referred to as?',
  'answer': "New York City is often referred to as 'The Big Apple.'"},
 {'question': 'What is the significance of Times Square in New York City?',
  'answer': "Times Square is a major commercial and entertainment hub known for its bright lights, theaters, and the annual New Year's Eve ball drop."},
 {'question': 'Which famous landmark is located in New York City?',
  'answer': 'The Statue of Liberty is one of the most famous landmarks located on Liberty Island in New York Harbor.'},
 {'question': 'What are the five boroughs of New York City?',
  'answer': 'The five boroughs of New York City are Manhattan, Brooklyn, Queens, The Bronx, and Staten Island.'}]

In [6]:
print(first_llm.render_prompt_printable(city_name="New York", num_records=5))


📝 CONSTRUCTED MESSAGES:

Role: user
Content:
Facts about city New York.

You are asked to generate exactly 5 records and please return the data in the following JSON format: 
[
    {
    "question": ""  //  (required),
    "answer": ""  //  (required)
    }
    ...
]

Required fields: question, answer

End of prompt



In [8]:
### Local model
first_llm = StructuredLLM(
    model_name="ollama/gemma3:1b",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
    model_kwargs={"temperature": 0.7},
)

first_response = await first_llm.run(city_name="New York", num_records=5)
first_response.data

2025-04-03 17:46:50 | INFO     | starfish.adapters.litellm_adapter | litellm_adapter.py:90 | Ensuring Ollama model gemma3:1b is ready...
2025-04-03 17:46:50 | INFO     | starfish.adapters.ollama_adapter | ollama_adapter.py:50 | Ollama server is already running
2025-04-03 17:46:50 | INFO     | starfish.adapters.ollama_adapter | ollama_adapter.py:132 | Found model gemma3:1b
2025-04-03 17:46:50 | INFO     | starfish.adapters.ollama_adapter | ollama_adapter.py:243 | Model gemma3:1b is already available
2025-04-03 17:46:50 | INFO     | starfish.adapters.litellm_adapter | litellm_adapter.py:99 | Model gemma3:1b is ready, making API call...


[{'question': 'What is the official population of New York City?',
  'answer': 'As of 2023, the estimated population of New York City is approximately 8.8 million people.'},
 {'question': 'Which borough is New York City located in?',
  'answer': 'New York City is divided into five boroughs: Manhattan, Brooklyn, Queens, The Bronx, and Staten Island.'},
 {'question': 'What is the name of the iconic landmark located in Central Park?',
  'answer': 'The iconic landmark in Central Park is the Bethesda Fountain.'},
 {'question': 'What is the highest point in New York City?',
  'answer': 'The highest point in New York City is the Top of the Rock Observation Deck at Rockefeller Center.'},
 {'question': "What is the name of New York City's famous Broadway theater?",
  'answer': 'Broadway is the name of New York City’s famous Broadway theater.'}]

In [9]:
from starfish.adapters.ollama_adapter import stop_ollama_server

await stop_ollama_server()

2025-04-03 17:47:04 | INFO     | starfish.adapters.ollama_adapter | ollama_adapter.py:264 | Stopping Ollama server...
2025-04-03 17:47:05 | INFO     | starfish.adapters.ollama_adapter | ollama_adapter.py:314 | Ollama server stopped successfully


True

In [5]:
### Local model
first_llm = StructuredLLM(
    model_name="hyperbolic/deepseek-ai/DeepSeek-V3-0324",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{'name': 'question', 'type': 'str'}, 
                   {'name': 'answer', 'type': 'str'}],
    model_kwargs={"temperature": 0.7}
)

first_response = await first_llm.run(city_name="New York", num_records=5)
first_response.data

[{'question': 'What is the nickname of New York City?',
  'answer': 'The Big Apple'},
 {'question': 'Which famous park is located in Manhattan?',
  'answer': 'Central Park'},
 {'question': 'What is the tallest building in New York City?',
  'answer': 'One World Trade Center'},
 {'question': 'Which subway system is the largest in the world by number of stations?',
  'answer': 'New York City Subway'},
 {'question': 'What iconic statue is located in New York Harbor?',
  'answer': 'Statue of Liberty'}]